In [ ]:
!pip install earthengine-api

In [ ]:
!earthengine authenticate

E0000 00:00:1750854842.195314     884 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1750854842.206007     884 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
Authenticate: Limited support in Colab. Use ee.Authenticate() or --auth_mode=notebook instead.
W0625 12:34:07.785487 137715955437568 _default.py:711] No project ID could be determined. Consider running `gcloud config set project` or setting the GOOGLE_CLOUD_PROJECT environment variable
To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/cloud-platform%20h

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
from google.colab import auth
auth.authenticate_user()

In [11]:
import ee, geemap
import os
ee.Authenticate()
ee.Initialize(project='ee-andavoluraviteja')

In [12]:
import folium

EE_TILES = 'https://earthengine.googleapis.com/map/{mapid}/{{z}}/{{x}}/{{y}}?token={token}'
print('Folium version: ' + folium.__version__)

Folium version: 0.19.7


In [13]:
from google.colab import userdata
import os

kaggle_credentials = userdata.get('KAGGLE_KEY')

!mkdir -p ~/.kaggle
with open('/root/.kaggle/kaggle.json', 'w') as f:
    f.write(kaggle_credentials)

!chmod 600 /root/.kaggle/kaggle.json

from kaggle.api.kaggle_api_extended import KaggleApi
api = KaggleApi()
api.authenticate()

print("Kaggle authentication successful!")

Kaggle authentication successful!


In [14]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf

In [20]:
import kagglehub

path = kagglehub.dataset_download("danofer/india-census")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/india-census


In [21]:
import os
import shutil

kaggle_download_path = '/kaggle/input/india-census'

drive_save_path = '/content/drive/My Drive'


os.makedirs(drive_save_path, exist_ok=True)

try:
    shutil.copytree(kaggle_download_path, drive_save_path, dirs_exist_ok=True)
    print(f"Dataset successfully copied from {kaggle_download_path} to {drive_save_path}")
except FileExistsError:
    print(f"Destination directory {drive_save_path} already exists and is not empty. Contents were likely merged.")
except Exception as e:
    print(f"An error occurred during copying: {e}")

Dataset successfully copied from /kaggle/input/india-census to /content/drive/My Drive


In [22]:
COLLECTION_ID = 'MODIS/MOD09GA_006_NDVI'
ndvi_ic = ee.ImageCollection(COLLECTION_ID)
india = (ee.FeatureCollection('FAO/GAUL/2015/level0')
           .filter(ee.Filter.eq('ADM0_NAME','India'))
           .geometry())
ndvi_2022 = (ndvi_ic
              .filterDate('2015-06-23', '2023-12-31')
              .mean()
              .multiply(0.00010)
              .clip(india))

In [23]:
Map = geemap.Map()
Map.centerObject(india, 5)
palette = ['Green','#c5e1a5','#8bc34a','#558b2f','#33691e']
Map.addLayer(ndvi_2022, {'min': 0.0, 'max': 1.0, 'palette': palette}, 'Mean NDVI 2022'
)
Map

Map(center=[22.22810924998663, 79.40094700957486], controls=(WidgetControl(options=['position', 'transparent_b…

In [24]:
import ee # Add import just in case, although it should be imported in the init cell
CHIRPS_ID = 'UCSB-CHG/CHIRPS/DAILY'
chirps_ic = ee.ImageCollection(CHIRPS_ID)

chirps_2022 = (chirps_ic
                 .filterDate('2015-06-23', '2023-12-31')
                 .sum()
                 .clip(india))

In [25]:
GRACE_ID = 'NASA/GRACE/MASS_GRIDS/LAND'
grace_ic = ee.ImageCollection(GRACE_ID)

grace_2022 = (grace_ic
                .filterDate('2015-06-23', '2023-12-31')
                .select('lwe_thickness')
                .mean()
                .clip(india))


In [26]:
LST_ID = 'MODIS/006/MOD11A2'
lst_ic = ee.ImageCollection(LST_ID)

lst_2022 = (lst_ic
              .filterDate('2015-06-23', '2023-12-31')
              .select('LST_Day_1km')
              .mean()
              .multiply(0.02)
              .subtract(273.15)
              .clip(india))


In [27]:
SMAP_ID = 'NASA_USDA/HSL/SMAP10KM_soil_moisture'
smap_ic = ee.ImageCollection(SMAP_ID)

smap_2022 = (smap_ic
               .filterDate('2015-06-23', '2023-12-31')
               .select('ssm')
               .mean()
               .clip(india))


In [28]:

combined = ndvi_2022.addBands([
    chirps_2022.rename('rainfall'),
    smap_2022.rename('soil_moisture'),
    grace_2022.rename('groundwater'),
    lst_2022.rename('temperature')
])


In [29]:
districts = ee.FeatureCollection("FAO/GAUL/2015/level1") \
               .filter(ee.Filter.eq('ADM0_NAME', 'India'))


In [30]:
def zonal_stats(image, collection, stat='mean', scale=500):
    reducer = {
        'mean': ee.Reducer.mean(),
        'sum': ee.Reducer.sum(),
        'median': ee.Reducer.median()
    }[stat]

    return image.reduceRegions(
        collection=collection,
        reducer=reducer,
        scale=scale
    )

In [31]:
ndvi_stats = zonal_stats(ndvi_2022, districts, 'mean', scale=500)


In [32]:
task = ee.batch.Export.table.toDrive(
    collection=ndvi_stats,
    description='export_ndvi_by_district',
    folder='earth_engine_exports',
    fileFormat='CSV'
)
task.start()


In [33]:
chirps_stats = zonal_stats(chirps_2022, districts, 'mean', scale=500)


In [34]:
task = ee.batch.Export.table.toDrive(
    collection=chirps_stats,
    description='export_rainfall_by_district',
    folder='earth_engine_exports',
    fileFormat='CSV'
)
task.start()

In [35]:
smap_stats = zonal_stats(smap_2022, districts, 'mean', scale=500)


In [36]:
task = ee.batch.Export.table.toDrive(
    collection=smap_stats,
    description='export_soilmoisture_by_district',
    folder='earth_engine_exports',
    fileFormat='CSV'
)
task.start()

In [37]:
grace_stats = zonal_stats(grace_2022, districts, 'mean', scale=500)


In [38]:
task = ee.batch.Export.table.toDrive(
    collection=grace_stats,
    description='export_groundwater_by_district',
    folder='earth_engine_exports',
    fileFormat='CSV'
)
task.start()

In [39]:
lst_stats = zonal_stats(lst_2022, districts, 'mean', scale=500)


In [40]:
task = ee.batch.Export.table.toDrive(
    collection=lst_stats,
    description='export_temperature_by_district',
    folder='earth_engine_exports',
    fileFormat='CSV'
)
task.start()